In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
tf.__version__

'2.15.0'

In [1]:
!apt install ffmpeg
!pip install -q spleeter
!pip install -q TeleBot
!pip install -q pyTelegramBotAPI
!pip install -q telegram
!pip install -q pydub
!pip install tensorflow==2.15.0

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438

In [2]:
!pip install tensorflow-estimator

In [3]:
!pip install tensorflow-io-gcs-filesystem

In [4]:
import os
import telebot
import telegram
import tensorflow as tf
import numpy as np
import pydub
from IPython.display import Audio
from spleeter.separator import Separator


In [4]:
#bot = telebot.TeleBot("6370453544:AAEI0zlqAp9HaPkuQv3o448Qs5YoYN1qSbs")
#model = tf.keras.models.load_model("audio_classification_model.h5")

In [5]:
bot = telebot.TeleBot("6370453544:AAEI0zlqAp9HaPkuQv3o448Qs5YoYN1qSbs")
model = tf.keras.models.load_model("audio_classification_model.h5")
model2 = tf.keras.models.load_model("singers_weights.h5")

@bot.message_handler(commands=["start"])
def send_welcome(message):
    bot.send_message(message.chat.id , "Hello " + message.from_user.first_name + "\nWelcome to Audio Recognizer bot🎙🎵🔊" )
    help(message)

@bot.message_handler(commands=['help'])
def help(message):
    my_keyboard= telebot.types.ReplyKeyboardMarkup(row_width=3)
    key1 = telebot.types.KeyboardButton("/Friends_voice_recognition")
    key2 = telebot.types.KeyboardButton("/Singers_voice_recognition")
    my_keyboard.row(key1)
    my_keyboard.row(key2)
    bot.send_message(message.chat.id ,"Choose one of the below keys :" , reply_markup=my_keyboard)


@bot.message_handler(commands=['Friends_voice_recognition'])
def mymessage(message):
  message1 = bot.send_message(message.chat.id , "Now send a voice ... ")
  bot.register_next_step_handler(message1 , friends_voice_recognizer)

@bot.message_handler(content_types=['voice'])
def friends_voice_recognizer(message):
  file_id = message.voice.file_id
  new_file = bot.get_file(file_id)
  file_path = bot.get_file(file_id).file_path
  downloaded_file = bot.download_file(file_path)
  try:
      with open("/content/data/voice.ogg", 'wb') as new_file:
          new_file.write(downloaded_file)
  except Exception as e:
      bot.reply_to(message, "Error saving file.")
      return

  audio = pydub.AudioSegment.from_file("/content/data/voice.ogg")
  audio_array = audio.get_array_of_samples()
  silent_chunks = pydub.silence.split_on_silence(audio , min_silence_len=2000 , silence_thresh=-45)
  result = sum(silent_chunks)
  result.export("/content/wav_data/voice.wav")
  audio = pydub.AudioSegment.from_file("/content/wav_data/voice.wav")
  chunks = pydub.utils.make_chunks(audio , 1000)
  for i , chunk in enumerate(chunks) :
      if len(chunk) >= 1000 :
          chunk.export(os.path.join("/content/dataset/z" , "fvoice.wav") , format="wav")


  test_data = tf.keras.utils.audio_dataset_from_directory(
    "/content/dataset",
    shuffle=False,batch_size=2,output_sequence_length= 48000,ragged= False,
    labels="inferred",label_mode="categorical",sampling_rate= None,seed=59
  )

  classes_labels = np.array(['abdollah', 'azra', 'davood', 'javad', 'khadijeh', 'kiana','maryam', 'matin','melika', 'mohammad',
                             'mohammad_parvari','mona', 'nahid', 'nima', 'omid', 'parisa','parsa', 'sadegh', 'sajedeh', 'shima', 'tara'])
  predictions = model.predict(test_data)
  probability = predictions
  predicted_class_index = np.argmax(probability)
  print(predicted_class_index)
  predicted_class = classes_labels[predicted_class_index]
  print(predicted_class)
  print(f'The owner of this voice is : {predicted_class}')
  bot.send_message(chat_id= message.chat.id , text=f'The owner of this voice is  :  {predicted_class}')





#~~~~~~~~~~~~~~~~ singers audio recognition ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
@bot.message_handler(commands=['Singers_voice_recognition'])
def mymessage2(message):
  message2 = bot.send_message(message.chat.id , "Now send a song...")
  bot.register_next_step_handler(message2 , singers_voice_recognition)

@bot.message_handler(content_types=['audio'])
def singers_voice_recognition(message):

  file_id = message.audio.file_id
  new_file = bot.get_file(file_id)
  file_path = bot.get_file(file_id).file_path
  downloaded_file = bot.download_file(file_path)
  try:
      with open("/content/data/audio.mp3", 'wb') as new_file:
          new_file.write(downloaded_file)
  except Exception as e:
      bot.reply_to(message, "Error saving file.")
      return


  audio_path = "/content/data/audio.mp3"
  separator = Separator("spleeter:2stems")
  prediction = separator.separate_to_file(audio_path, "/content/singers_vocals")


  audio = pydub.AudioSegment.from_file("/content/singers_vocals/audio/vocals.wav")
  audio_array = audio.get_array_of_samples()
  silent_chunks = pydub.silence.split_on_silence(audio , min_silence_len=100 , silence_thresh=-50)
  result = sum(silent_chunks)
  result.export("/content/wav_data/audio.wav")
  audio = pydub.AudioSegment.from_file("/content/wav_data/audio.wav")
  chunks = pydub.utils.make_chunks(audio , 1000)
  for i , chunk in enumerate(chunks) :
      if len(chunk) >= 1000 :
          chunk.export(os.path.join("/content/dataset_2/x" , "final_audio.wav") , format="wav")


  test_data = tf.keras.utils.audio_dataset_from_directory(
    "/content/dataset_2",
    shuffle=False,batch_size=2,output_sequence_length= 48000,ragged= False,
    labels="inferred",label_mode="categorical",sampling_rate= None,seed=59
  )

  classes_labels = np.array(['chaartar', 'chavoshi', 'sirvan', 'xaniar', 'yegane'])
  predictions = model2.predict(test_data)
  probability = predictions
  predicted_class_index = np.argmax(probability)
  print(predicted_class_index)
  predicted_class = classes_labels[predicted_class_index]
  print(predicted_class)
  print(f'The owner of this song is : {predicted_class} ')
  bot.send_message(chat_id= message.chat.id , text=f'The owner of this song is  :  {predicted_class}')


bot.infinity_polling()

Found 1 files belonging to 1 classes.
1/1 [==============================] - 0s 239ms/step
5
kiana
The owner of this voice is : kiana


2023-12-27 10:32:02,595 (__init__.py:968 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2023-12-27 10:32:02,599 (__init__.py:970 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
